# Federal Reserve Bank - Data Collection

<b> Author: </b> Derek A Maier | maierd@canisius.edu

<hr>

## Scope:

The scope of this workbook includes the following:

- Data Collection from the Federal Reserve data site
- Data Manipulation / Cleansing for Analysis
- Storage of the data collection above

<hr> 

## Assumptions: 

The development and execution of this workbook / analysis is based on the following assumptions (<i>check marks indicate the assumption has been met</i>):

- <b> ✓ </b> Ethical ability to scrape the FRB Site
- <b> ✓ </b> Data Availability of Company data for prior two years 

<hr>

## References:

The data was provided by the following base site: https://www.federalreserve.gov/releases/lbr/

<hr>

## 1.a. Import Libraries & Set Global Variables

In [1]:
# Imports
import os
import pickle
import pandas as pd
from datetime import datetime

from dam.scrape import Engine # NOTE: This is a custom package I created for ethical hacking, simply replace with a GET request

In [2]:
# Global variables
data_dir = 'data/'
data_file = os.path.join(data_dir, 'commercial_bank_data.xlsx')
base_url = "https://www.federalreserve.gov/releases/lbr/"

## 1.b. Utility Functions to Parse the FRB Site(s)

In [3]:
# Create all utility functions for parsing the FRB Soup Objects

# Step 1. Gather all date specific page URLs
def parse_base(url):

    """
    Description:
    ------------
    Parses all of individual date specific links & reports for the data collection step
    
    Params:
    ------------
    url : (str)
        Represents the base url to scrape, or defaults to the static URL
        
    Returns
    ------------
    data : (list)
        Represents a list of dictionaries with the URL and Date of the report 
    
    """
     
    data = None    
    engine = Engine()
    
    try:
        soup = engine.soup_request(url)
        main_div = soup.find("div", attrs={"id": "content"}) # Main container is second out of three
        data = [base_url + val for val in [node["href"] for node in main_div.find_all("a")][2:-1] if val.split('.')[1] == 'htm']
        
    except:
        pass
    
    return data


# Step 2. Parse the Date specific Sites
def parse_data(url):
    
    """
    Description:
    ------------
    Parses all of commercial bank table data from the quarter specific URL site
    
    Params:
    ------------
    url : (str)
        Represents the base url to scrape, or defaults to the static URL
        
    Returns
    ------------
    data : (list)
        Represents a list of dicts with all data collected from the given URL site
    
    """
    
    data = []    
    engine = Engine()
    
    try:
        soup = engine.soup_request(url)
        table = soup.find_all('table')[1] # Main table with data
        trs = table.find_all("tr")[1:]

        for i in range(len(trs)):
            data.append({
                'bank_holding_name': trs[i].find_all("td")[0].text,
                'national_rank': trs[i].find_all("td")[1].text,
                'bank_id': trs[i].find_all("td")[2].text,
                'bank_location': trs[i].find_all("td")[3].text,
                'charter': trs[i].find_all("td")[4].text,
                'consolidated_assets': trs[i].find_all("td")[5].text,
                'domestic_assets': trs[i].find_all("td")[6].text,
                'pct_domestic_assets': trs[i].find_all("td")[7].text,
                'pct_cumulative_assets': trs[i].find_all("td")[8].text,
                'no_domestic_branches': trs[i].find_all("td")[9].text,
                'no_foreign_branches': trs[i].find_all("td")[10].text,
                'ibf': trs[i].find_all("td")[11].text,
                'as_of_dt': url.split('/')[-2].replace('current', '20200331').strip()
            })
        
    except:
        pass
    
    return data
    

## 1.c. Gather & Dump the Base URL Page Links 

In [4]:
# Set links to scrape to a variable using our utility function
links_to_scrape = parse_base(base_url)

In [20]:
links_to_scrape[:10]

['https://www.federalreserve.gov/releases/lbr/current/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20191231/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20190930/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20190630/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20190331/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20181231/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20180930/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20180630/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20180331/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20171231/default.htm']

## 1.d. Scrape all Links above and Store Results

In [6]:
# Create an empty list to store all results to 
commercial_data = []

for idx, url in enumerate(links_to_scrape):
    print(f'Collecting Data for Link No. {idx}')
    commercial_data += parse_data(url)

In [21]:
print(f'The total number of records returned for our dataset : {len(commercial_data)}')
print("="*70)
commercial_data[0:5]

The total number of records returned for our dataset : 98354


[{'bank_holding_name': 'JPMORGAN CHASE BK NA/JPMORGAN CHASE & CO ',
  'national_rank': '1 ',
  'bank_id': '852218 ',
  'bank_location': 'COLUMBUS, OH ',
  'charter': 'NAT ',
  'consolidated_assets': '2,690,959 ',
  'domestic_assets': '1,990,017 ',
  'pct_domestic_assets': '74 ',
  'pct_cumulative_assets': '15 ',
  'no_domestic_branches': '4,971 ',
  'no_foreign_branches': '33 ',
  'ibf': 'Y ',
  'as_of_dt': '20200331'},
 {'bank_holding_name': 'BANK OF AMER NA/BANK OF AMER CORP ',
  'national_rank': '2 ',
  'bank_id': '480228 ',
  'bank_location': 'CHARLOTTE, NC ',
  'charter': 'NAT ',
  'consolidated_assets': '2,031,940 ',
  'domestic_assets': '1,925,005 ',
  'pct_domestic_assets': '95 ',
  'pct_cumulative_assets': '26 ',
  'no_domestic_branches': '4,239 ',
  'no_foreign_branches': '26 ',
  'ibf': 'Y ',
  'as_of_dt': '20200331'},
 {'bank_holding_name': 'WELLS FARGO BK NA/WELLS FARGO & CO ',
  'national_rank': '3 ',
  'bank_id': '451965 ',
  'bank_location': 'SIOUX FALLS, SD ',
  'chart

## 1.e. Convert to DataFrame and Dump to Disk

In [18]:
# Convert to DataFrame
df = pd.DataFrame(commercial_data)

# Let's also create columns for the reporting year & month
df['as_of_year'] = df['as_of_dt'].apply(lambda x: x[0:4])
df['as_of_mth'] = df['as_of_dt'].apply(lambda x: x[4:6])
df['as_of_period'] = df['as_of_mth'] + '-' + df['as_of_year']

df.head()

,bank_holding_name,national_rank,bank_id,bank_location,charter,consolidated_assets,domestic_assets,pct_domestic_assets,pct_cumulative_assets,no_domestic_branches,no_foreign_branches,ibf,as_of_dt,as_of_year,as_of_mth,as_of_period
0,JPMORGAN CHASE BK NA/JPMORGAN CHASE & CO,1,852218,"COLUMBUS, OH",NAT,"2,690,959","1,990,017",74,15,"4,971",33,Y,20200331,2020,03,03-2020
1,BANK OF AMER NA/BANK OF AMER CORP,2,480228,"CHARLOTTE, NC",NAT,"2,031,940","1,925,005",95,26,"4,239",26,Y,20200331,2020,03,03-2020
2,WELLS FARGO BK NA/WELLS FARGO & CO,3,451965,"SIOUX FALLS, SD",NAT,"1,763,696","1,715,842",97,35,"5,416",11,Y,20200331,2020,03,03-2020
3,CITIBANK NA/CITIGROUP,4,476810,"SIOUX FALLS, SD",NAT,"1,632,405","971,079",59,44,697,157,Y,20200331,2020,03,03-2020
4,U S BK NA/U S BC,5,504713,"CINCINNATI, OH",NAT,"533,129","522,496",98,47,"2,832",1,N,20200331,2020,03,03-2020


In [9]:
# Explore some quick stats for the health of the data
df.describe()

,bank_holding_name,national_rank,bank_id,bank_location,charter,consolidated_assets,domestic_assets,pct_domestic_assets,pct_cumulative_assets,no_domestic_branches,no_foreign_branches,ibf,as_of_dt
count,98354,98354,98354,98354,98354,98354,98354,98354,98354,98354,98354,98354,98354
unique,4331,1854,3182,2156,3,11269,11274,57,74,1282,80,3,56
top,BREMER BK NA/OTTO BREMER FOUNDATION,1036,591320,"NEW YORK, NY",SNM,.,.,100,99,6,0,N,20200331
freq,276,62,189,1413,55446,575,575,96009,18513,6784,95983,95556,1863


In [19]:
# Dump to disk for storage and next stage in the project
df.to_excel(data_file, index=False)